In [53]:
using Revise
using Random
using Printf
using LinearAlgebra
using Statistics
import Tensor_FixedSeedMC as TCIMC

In [54]:
function asian_barrier_option(T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS)
    tmp1 = mu * T / N_STEPS
    tmp2 = exp(-r * T)
    tmp3 = sqrt(T / N_STEPS)
    d_s = zeros(Float64, N_PATHS)
    
    for i in 1:N_PATHS
        s_curr = S0
        running_average = 0.0
        
        for n in 1:N_STEPS
            # Update the current price
            s_curr += tmp1 * s_curr + sigma * s_curr * tmp3 * d_normals[i + (n - 1) * N_PATHS]
            
            # Update the running average
            running_average += (s_curr - running_average) / n
            
            # Check the barrier
            if running_average <= B
                break
            end
        end
        
        # Calculate payoff
        payoff = max(running_average - K, 0)
        
        # Discounted payoff
        d_s[i] = tmp2 * payoff
    end
    mean_ = mean(d_s)
    #abs_ = std(d_s)
    standard_err =  std(d_s) / sqrt(N_PATHS)
    # 
    return mean(d_s), standard_err
end


asian_barrier_option (generic function with 1 method)

In [55]:
# Example usage
N_STEPS = 365
#N_PATHS = 50000000
N_PATHS = 819200
T = 1.0
K = 110.0
B = 100.0
S0 = 120.0
sigma = 0.35
mu = 0.1
r = 0.05
d_normals = randn(Float64, N_STEPS * N_PATHS)  # Standard normal random numbers

mean_ds, err_ds = asian_barrier_option(T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS)

# Display a sample of results
@printf("Sample of computed option prices: %0.4f\n", mean_ds)
@printf("Sample of computed option errors: %0.4f\n", err_ds)

Sample of computed option prices: 18.7193
Sample of computed option errors: 0.0235


In [13]:
tolerance = 1e-3
maxval = 20
tolerance * maxval 

0.02

In [10]:
using JLD2
JLD2.@load "price_matrix_asianbarrier.jld2" price_matrix

KeyError: KeyError: key "price_matrix" not found

In [37]:
len = 100
d = 2
num_comb = 100
random_combinations = Tensor_FixedSeedMC.generate_random_combinations(len, d, num_comb);

In [ ]:
S0_values = range(90, stop=120, length=101)[1:end-1]
implied_vol_values = range(0.15, stop=0.25, length=101)[1:end-1]

In [ ]:
mc = []
for i in random_combinations
    S0 = S0_values[i[1]]
    sigma = implied_vol_values[i[2]]
    price, _ = asian_barrier_option(T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS)
    #println("S0: ", S0, ", sigma: ", sigma, ", price: ", price)
    push!(mc, price)
end
mc

In [40]:
using JLD2
JLD2.@save "mc.jld2" mc